## Trained Embedded Layer
- created: 23 MArch 2020

In [38]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras
from joblib import dump, load
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tf.random.set_seed(42)

## Load Data 

In [25]:
def prep_data(data_dir,prep_folder):

    features = np.load(join(data_dir,prep_folder,'features.npy'))
    
    with open(join(data_dir,'syllable_name.txt')) as f:
        label = np.array([char for line in f for word in line.split(',') for char in word.strip()])
        
    print('Predict Data {} {}'.format(features.shape, label.shape))
    return features, label

In [26]:
DATA_PATH = '../../data/d_records/all/'
PREP_FOLDER = 'prep_data_13_2'
features, label = prep_data(DATA_PATH, PREP_FOLDER)

Predict Data (1296, 24, 39) (1296,)


In [27]:
set(label)

{'7', 'E', 'M', 'O', 'a', 'e', 'i', 'o', 'u'}

### One-hot label 

In [39]:
enc = load('onehot.joblib') 

### Defined (Between) Embedding Model

In [28]:
model = models.load_model('between_embedded_32.hdf5')

In [29]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 39)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 24, 32)            9216      
_________________________________________________________________
flatten (Flatten)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 14)                10766     
Total params: 19,982
Trainable params: 19,982
Non-trainable params: 0
_________________________________________________________________


### Predict

In [46]:
pred = model.predict(features)

In [47]:
pred_trans=enc.inverse_transform(pred)

In [48]:
pred_trans = pred_trans.reshape((pred_trans.shape[0]*pred_trans.shape[1],))
pred_trans

array(['a', 'i', 'A', ..., 'o', 'O', 'a'], dtype='<U2')

In [49]:
label

array(['a', 'i', 'a', ..., 'o', 'O', 'O'], dtype='<U1')

In [54]:
false_pred_trans = pred_trans[pred_trans != label]

In [56]:
false_pred_trans.shape

(841,)

In [58]:
unique, counts = np.unique(false_pred_trans, return_counts=True)
dict(zip(unique, counts))

{'2': 63,
 '9': 22,
 '@': 21,
 'A': 154,
 'E': 42,
 'E:': 97,
 'O': 13,
 'U': 76,
 'a': 68,
 'e': 18,
 'i': 51,
 'o': 111,
 'u': 4,
 'y': 101}

In [59]:
false_label = label[pred_trans != label]
unique, counts = np.unique(false_label, return_counts=True)
dict(zip(unique, counts))

{'7': 144,
 'E': 138,
 'M': 144,
 'O': 57,
 'a': 139,
 'e': 66,
 'i': 22,
 'o': 9,
 'u': 122}